# Census Data

## Grabbing Income Variables

Exploring the variables available from census data first, using detailed tables.  A lot of variables here, need to narrow it down and still possibly manage at scale.

In [4]:
import pandas as pd

In [53]:
variables = pd.read_html('https://api.census.gov/data/2018/acs/acs5/variables.html')[0]

In [54]:
variables.head()

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANHH,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
1,AIHHTL,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
2,AIRES,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
3,AITS,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN
4,ANRC,Geography,NaN,not required,NaN,0,(not a predicate),NaN,NaN


Let's try to look at drinking demand in relation to income concepts.  This may miss a few, but is still a large group.  No Puerto Rico is an assumption from working with Iowa, could've just left it but the difference between more a few groups I guess.  Anyways, left out

In [55]:
income_concepts = [concept for concept in variables.Concept.unique() if type(concept)==str 
                 and 'INCOME' in concept 
                 and ';' not in concept
                 and 'PUERTO RICO' not in concept]
print(len(income_concepts))
income_concepts

179


['RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS',
 'PLACE OF BIRTH BY INDIVIDUAL INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) IN THE UNITED STATES',
 'MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) BY PLACE OF BIRTH IN THE UNITED STATES',
 'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY INDIVIDUAL INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) FOR CURRENT RESIDENCE IN THE UNITED STATES',
 'MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) BY GEOGRAPHICAL MOBILITY IN THE PAST YEAR FOR CURRENT RESIDENCE IN THE UNITED STATES',
 'GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY INDIVIDUAL INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) FOR RESIDENCE 1 YEAR AGO IN THE UNITED STATES',
 'MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS) BY GEOGRAPHICAL MOB

In [84]:
agg_mask = variables.Concept.map(lambda concept: 'MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS' in concept if type(concept)==str else False)
variables[agg_mask].head()

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
12169,B19113_001E,Estimate!!Median family income in the past 12 ...,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,not required,"B19113_001M, B19113_001MA, B19113_001EA",0,int,B19113,NaN
12170,B19113A_001E,Estimate!!Median family income in the past 12 ...,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,not required,"B19113A_001M, B19113A_001MA, B19113A_001EA",0,int,B19113A,NaN
12171,B19113B_001E,Estimate!!Median family income in the past 12 ...,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,not required,"B19113B_001M, B19113B_001MA, B19113B_001EA",0,int,B19113B,NaN
12172,B19113C_001E,Estimate!!Median family income in the past 12 ...,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,not required,"B19113C_001M, B19113C_001MA, B19113C_001EA",0,int,B19113C,NaN
12173,B19113D_001E,Estimate!!Median family income in the past 12 ...,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN...,not required,"B19113D_001M, B19113D_001MA, B19113D_001EA",0,int,B19113D,NaN


If we wanted to grab these, what groups would we need?  Not that many that I'm discouraged from changing previous code to look at group id's and inspecting groups by doing THIS

In [56]:
income_groups = variables[variables.Concept.isin(income_concepts)].Group.unique()
len(income_groups)

179

## Using the Convenient API Wrapper

Shoutouts to the maintainers, repo [here](https://github.com/datamade/census)

In [2]:
from census import Census
from us import states

It's here if you want it

In [52]:
c = Census('aae12ac68f626fd88f1c8293d164e7e52c0cedd0', year=2018)

In [65]:
test_result = c.acs5.get(['NAME', f'group({income_groups[2]})'],
                         {'for':'county:*', 'in':f'state:{states.IA.fips}'})

In [66]:
len(test_result)

99